In [0]:
#@title All import statements
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta
from random import random
from numpy import array
from numpy import cumsum
from numpy import sum
from google.colab import files
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.layers import Bidirectional
from keras.preprocessing.text import Tokenizer
from string import ascii_lowercase
from operator import add
from itertools import chain
from decimal import *
import keras.utils as ku 
from keras.preprocessing.sequence import pad_sequences
from google.colab import files
import pandas as pd
import numpy as np
import csv
import ctypes.util
import magenta.music as mm
import magenta
import math
import operator
from magenta.protobuf import music_pb2
# Hack to allow python to pick up the newly-installed fluidsynth lib. 
# This is only needed for the hosted Colab environment.
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library


getcontext().prec = 6


In [0]:
#@title Upload File 
files.upload()

Saving Ninnukori.csv to Ninnukori.csv
Saving Jalajakshi.csv to Jalajakshi (2).csv


{u'Jalajakshi.csv': '0, 0, Header, 1, 2, 96\n1, 0, Start_track\n1, 0, Tempo, 1000000\n1, 0, Time_signature, 4, 2, 24, 8\n1, 0, End_track\n2, 0, Start_track\n2, 0, Title_t, "SwarPlug [64bit] 1"\n2, 0, Note_on_c, 0, 16, 100\n2, 48, Note_off_c, 0, 16, 64\n2, 48, Note_on_c, 0, 14, 100\n2, 96, Note_off_c, 0, 14, 64\n2, 96, Note_on_c, 0, 12, 100\n2, 144, Note_off_c, 0, 12, 64\n2, 192, Note_on_c, 0, 11, 100\n2, 240, Note_off_c, 0, 11, 64\n2, 240, Note_on_c, 0, 12, 100\n2, 288, Note_off_c, 0, 12, 64\n2, 288, Note_on_c, 0, 14, 100\n2, 336, Note_off_c, 0, 14, 64\n2, 336, Note_on_c, 0, 16, 100\n2, 384, Note_off_c, 0, 16, 64\n2, 384, Note_on_c, 0, 14, 100\n2, 432, Note_off_c, 0, 14, 64\n2, 432, Note_on_c, 0, 14, 100\n2, 480, Note_off_c, 0, 14, 64\n2, 480, Note_on_c, 0, 12, 100\n2, 528, Note_off_c, 0, 12, 64\n2, 528, Note_on_c, 0, 11, 100\n2, 576, Note_off_c, 0, 11, 64\n2, 624, Note_on_c, 0, 12, 100\n2, 672, Note_off_c, 0, 12, 64\n2, 672, Note_on_c, 0, 14, 100\n2, 720, Note_off_c, 0, 14, 64\n2, 720

In [0]:
#@title All Variables
#Tune Variables
tuneTimeSlice = 11
tuneOutputSpan = 3
maxOutputSpan = 5
tuneTemperature = 4
variety = 5
tuneReverse = True
transpose = 0


holdModel = True

#Remember One Step = 24 MIDI Clocks
#Also Remember One count = 4 x 24 = 96 MIDI Clocks
#Also Also Remember 4 counts = 4 x 96 = 384 MIDI Clocks

#Motif variables
NumberOfMotifsOfSizeN = 4
maxSizeOfMotif = 5
motifTimeSlice = 11
motifOutputSpan = 1
maxOutputSpan = 5
motifTemperature = 4
variety = 5
motifReverse = True
# fourTimeSignature = True
# stepsPerBeat = 4
# if stepsPerBeat == 4:
#   fourTimeSignature = True
# else:
#   fourTimeSignature = False
# if stepsPerBeat == 4:
#   clocksPerStep = 24
#   stepsPerBar = 16
# elif stepsPerBeat == 3:
#   clocksPerStep = 32
#   stepsPerBar = 12
  
  
  
  
#Compose variables
endNoteSize = 2
numberOfEndNotes = 1
numberOfPhrases = 4
outfilename = 'Out.csv'



inputFileName = 'Jalajakshi.csv'
transpose = 0


In [0]:
#@title CSV Processing
#We need to remove the first 7 rows of the csv along with all other channels that don't contain melody's midi notes
channel = 0
rowIndex = 0 
tempo = 0 
count =0
NewIndex = 0
countStart = 0
with open(outfilename,'w') as outputFile:
    csvWriter  = csv.writer(outputFile)
    with open(inputFileName) as csvfile:
         csvReader = csv.reader(csvfile)
         for row in csvReader:
              rowIndex += 1
              if row[2] == ' Start_track':
#                 countStart += 1
#                 if countStart ==1:
                  channel = row[0]
              if row[2] == ' Note_on_c':
                NewIndex = rowIndex
              if row[2]==' Tempo':
                  count +=1
                  if count == 1:  
                    tempo = 60000000/int(row[3])
                    csvWriter.writerow([int(tempo)]+[0]+[0])
              if row[2]==' Header':
                clocksPerBeat = int(row[5])
              if rowIndex >= NewIndex and row[0] == channel and row[2] == ' Note_on_c' and int(row[5]) > 0:
                  csvWriter.writerow([row[1]] + [row[4]] + [row[5]])
    outputFile.close()
    
    


my_cols = ["Clock","note","Velocity"]
dataFrame = pd.read_csv('Out.csv' , names=my_cols, engine='python').iloc[1:]




#We find the time signature of the song and then quantisize the notes
fourStepError = 0
fourStepErrorRows = []
threeStepError = 0
threeStepErrorRows = []

fourSteps = clocksPerBeat / 4
threeSteps = clocksPerBeat / 3
for i in range(min(100,len(dataFrame))):
  temp = min(dataFrame.iloc[i][0] % fourSteps,fourSteps - (dataFrame.iloc[i][0] % fourSteps))
  if temp != 0:
    fourStepError += temp
    fourStepErrorRows += [i]
  temp = min(dataFrame.iloc[i][0] % threeSteps,threeSteps - (dataFrame.iloc[i][0] % threeSteps))
  if temp != 0:
    threeStepError += temp
    threeStepErrorRows += [i]
if fourStepError <= threeStepError:
  clocksPerStep = clocksPerBeat / 4
  clocksPerBar = clocksPerBeat * 4
  stepsPerBeat = 4
  stepsPerBar = 16
  error = fourStepError
  errorRows = fourStepErrorRows
else:
  clocksPerStep = clocksPerBeat / 3
  clocksPerBar = clocksPerBeat * 4
  stepsPerBeat = 3
  stepsPerBar = 12
  error = threeStepError
  errorRows = threeStepErrorRows
  
  
  
if error != 0:
  clock = dataFrame['Clock'].iloc[:]
  for i in errorRows:
    if dataFrame.iloc[i][0] % clocksPerStep != 0:
      if (dataFrame.iloc[i][0] % clocksPerStep)  <  (clocksPerStep / 2):
        clock[i] =  dataFrame.iloc[i][0] - (dataFrame.iloc[i][0] % clocksPerStep)
      else:
        clock[i] = dataFrame.iloc[i][0] + (clocksPerStep - (dataFrame.iloc[i][0] % clocksPerStep))
  dataFrame['Clock'] = clock
  
  

#Remove Harmonies
deletableRows = []
i = 0
while i < len(dataFrame) - 1:
  if dataFrame.iloc[i][0] == dataFrame.iloc[i+1]['Clock']:
    j = i + 1
    while dataFrame.iloc[i]['Clock'] == dataFrame.iloc[j]['Clock'] and j < len(dataFrame)-1 :
      j += 1
    velocities = dataFrame.iloc[i:j]['Velocity']
    if (len(set(velocities))==1):
      notes = dataFrame.iloc[i:j]['note']
      for k in range(i,j):
        if dataFrame.iloc[k]['note'] != max(notes):
          deletableRows += [k]
          i += 1
    else:
      for k in range(i,j):
        if dataFrame.iloc[k]['Velocity'] != max(velocities):
          deletableRows += [k]
          i += 1
  i += 1
dataFrame = dataFrame.drop(dataFrame.index[deletableRows])




#Calculate note lengths
noteLengths = []
offBy = []
for i in range(len(dataFrame) - 1):
  noteLength = (dataFrame.iloc[i + 1]['Clock'] - dataFrame.iloc[i][0]) / clocksPerStep
  off = int((dataFrame.iloc[i][0] % clocksPerBar)/clocksPerStep)
  if int(dataFrame.iloc[i + 1]['Clock']/clocksPerBar) > int(dataFrame.iloc[i][0]/clocksPerBar):
    noteLength = stepsPerBar - off
  noteLengths += [int(noteLength)]
  offBy += [off]
noteLengths += [int((clocksPerStep * 4 - (dataFrame.iloc[-1]['Clock']  % clocksPerStep * 4))/clocksPerStep)]
offBy += [int((dataFrame.iloc[i][0] % clocksPerBar)/clocksPerStep)]
dataFrame['Length'] = noteLengths
dataFrame['Off By'] = offBy
dataFrame = dataFrame[dataFrame['Length'] != 0]


 



#Transpose notes
notes = []
for i in range(0,len(dataFrame)):
  notes += [dataFrame.iloc[i]['note'] + (0 - transpose)]
dataFrame['note'] = notes




#Get a list of bars
bars = []
notes = []
endingSpace = []

start = 0
t = dataFrame.iloc[0]['Off By']
for i in range(0,len(dataFrame) - 1):
  t += dataFrame.iloc[i]['Length']
  if t % stepsPerBar == 0:
    t = 0
    bars += [list(dataFrame.iloc[start:i+1]['Length'])]
    endingSpace += [(dataFrame.iloc[i+1]['Clock'] - dataFrame.iloc[i][0])/ clocksPerStep ]

    notes += [list(dataFrame.iloc[start:i+1]['note'])]
    start = i + 1
    if dataFrame.iloc[start]['Off By'] != 0:
      t += dataFrame.iloc[start]['Off By']

      
maxNote = max([max(n) for n in notes])
# maxNote = sum(maxNote)/len(maxNote)
      
# octaveShift = (int)(maxNote/12) - 1
octaveShift = (int)(maxNote/12) - 2
shift = (octaveShift * 12)

notes=[(np.array(x) - shift).tolist() for x in notes]

      
      
      
if holdModel == True:
  try:
    bars += holdBars
    notes += holdNotes
    endingSpace += holdEndingSpace
  except NameError:
    None

      
      
holdNotes = notes[:]
holdBars = bars[:]
holdEndingSpace = endingSpace[:]






In [0]:
#@title Bar Pattern Approach 2

def sum2(arr):
  x = []
  for i in arr:
    x += i
  return x

barsCopy = []
notesCopy = []
endingSpaceCopy = []
averageEndingNoteLength = 2
start = 0
for i in range(len(bars)):
  npArray = np.array(bars[i])
  lengthyNotesPositions = np.where(npArray > 4)[0]
  barLength =  i - start
  if lengthyNotesPositions.size > 0 :
    endNoteScore = (lengthyNotesPositions[-1] / len(npArray)) * barLength
    if bars[i][lengthyNotesPositions[-1]] != bars[i][-1] and (endNoteScore) >= 0.5:
      barsCopy += [bars[i][:lengthyNotesPositions[-1] + 1]]
      barsCopy += [bars[i][lengthyNotesPositions[-1] + 1:]]
      notesCopy += [notes[i][:lengthyNotesPositions[-1] + 1]]
      notesCopy += [notes[i][lengthyNotesPositions[-1] + 1:]]
      endingSpaceCopy += [bars[i][lengthyNotesPositions[-1]],endingSpace[i]]
    else:
      barsCopy += [bars[i]]
      notesCopy += [notes[i]]
      endingSpaceCopy += [endingSpace[i]]
  else:
    barsCopy += [bars[i]]
    notesCopy += [notes[i]]
    endingSpaceCopy += [endingSpace[i]]
  if endingSpace[i] > averageEndingNoteLength:
    start = i + 1
    

averageEndingNoteLength = 2
Phrases = []
endNotes = []
phrasesNotes = []
start = 0
for i in range(len(barsCopy)):
  if endingSpaceCopy[i] > averageEndingNoteLength:
    Phrases += [sum2(barsCopy[start:i+1])]
    phrasesNotes += [sum2(notesCopy[start:i+1])]
    endNotes += [phrasesNotes[-1][-1]]
    start = i + 1
    
    
phrasesCopy = []
for i in Phrases:
  if len(i) != 1 :
    phrasesCopy += [i] 
    
    
arr = [sum(x)/stepsPerBar for x in phrasesCopy]
averageBarLength2 = sum(arr)/len(arr)

In [0]:
#@title User input
g = raw_input("Enter phrase required")
userPhrase = g.split(',')
userPhrase = np.array(userPhrase).astype(int)
LenOfPhrase = np.size(userPhrase)
userEndNote = userPhrase[LenOfPhrase-1]


Enter phrase required16,14,16,14,12


In [0]:
#@title Common Functions

def nMostProbablePredictions(num,pred):
  predCopy = pred
  selectedPredictions = []
  for j in range(0,num):
    max = 0
    maxIndex = 0
    for i in range(0,len(predCopy)):
      if predCopy[i] > max:
        max = predCopy[i]
        maxIndex = i
    predCopy[maxIndex] = -1
    selectedPredictions = selectedPredictions + [maxIndex]
  return selectedPredictions

In [0]:
#@title All Tune model Functions

#Inputs : inputLines - List of Note sequences, where each note sequence is a phrase in the song
#Output : n_gram_sequences : n_gram_sequences required for input for LSTMS
def generateTuneNGramSequence(inputLines):
    ## convert data to sequence of tokens 
    n_gram_sequences = []
    for line in inputLines:
      if tuneReverse == True:
        line = np.flip(line)
      for i in range(1, len(line)):
          n_gram_sequence = line[:i+1]
          n_gram_sequences.append(n_gram_sequence)
    return n_gram_sequences


  
  
#Inputs : input_sequences : List of Notes sequences of varying lenght
#Outputs :  X : Note sequences of length equal to tuneTimeSlice - tuneOutputSpan pre padded with zeros
#           y : Set of output notes for each X encoded in one hot encoding format
#           intervals : Set of output notes for each X in readable format
#           tokenizer : word index mapping set of output notes to their uniquely assigned labels
#           classes : Number of possible combination of Output Notes that for y
def generate_padded_sequences_notes(input_sequences):
  input_sequences = np.array(pad_sequences(input_sequences, maxlen=tuneTimeSlice, padding='pre'))
  X = []
  intervals = []
  for i in range(0,len(input_sequences)):
    if np.count_nonzero(input_sequences[i,:-tuneOutputSpan]) != 0:
      X += [list(input_sequences[i,:-tuneOutputSpan])]
      interval = str(input_sequences[i,-tuneOutputSpan])
      for j in range(-tuneOutputSpan + 1 ,0):
          interval += '-' + str(input_sequences[i,j])
      intervals += [interval]
  X = np.array(X)
  tokenizer = Tokenizer(filters = '_')
  tokenizer.fit_on_texts(intervals)
  y = []
  for i in intervals:
    y = y + [tokenizer.word_index[i]]
  classes = np.max(y)+1
  y = ku.to_categorical(y,classes)
  return X,y,intervals,tokenizer,classes



swaras = {
    "1":"Sa",
    "2":"Ri1",
    "3":"Ri2",
    "4":"Ga1",
    "5":"Ga2",
    "6":"Ma1",
    "7":"Ma2",
    "8":"Pa",
    "9":"Da1",
    "10":"Da2",
    "11":"Ni1",
    "12":"Ni2"
}

#Inputs :  i : Index of note
#Outputs : swara : Corresponding swara in Sa Ri Ga .. format
def IndexToSwara(i):
  if int(i) == 0:
    return ' '
  i = (int(i) % 12) + 1
  return swaras[str(i)]


#Inputs :  i : Array of note indices 
#Outputs : swara : Corresponding array of swaras in Sa Ri Ga .. format
def arrayToSwara(arr):
  swarArray = []
  for i in arr:
    swarArray = swarArray + [IndexToSwara(i)]
  return swarArray


#Inputs :  num : Number of required max probable notes
#          pred : LSTM's output
#Outputs : a : List of notes with max probability


#Inputs :  interval : Set of output notes of lstm labelled according to tokenizer's word index 
#Outputs : array : array of output note indices
def intervalToIndex(interval):
  array = []
  for word,index in tokenizertune.word_index.items():
            if index == interval:
              for index in word.split('-'):
                 array += [int(index)]
  return array



In [0]:
#@title All End Note Functions

#Input : Notes - Array of one end note,length -length of how many endNotes needed.
#Output : output - Array of indices of end notes
def getEnding(Notes,length):
  output=[]
  for i in range(0,length):
    predArray =  list(outmodel.predict_proba([[list(Notes)]], verbose=0)[0])
    pred = nMostProbablePredictions(variety,predArray)[0]#[int(random() * tuneTemperature)]
    output = np.append(output,pred)
    Notes = np.append(Notes,pred)
    Notes = np.delete(Notes,[0])
  return np.array(output,dtype=int)


#Inputs : input_sequences : List of end note sequences of varying lenght
#Outputs :  input_sequencse : End note sequences  of length equal to tuneTimeSlice pre padded with zeros
#           X : End note sequences  of length equal to tuneTimeSlice - tuneOutputSpan pre padded with zeros
#           y : Set of output end notes for each X encoded in one hot encoding format
def generate_padded_sequences_end(input_sequences):
#  classes = np.amax(input_sequences)
  input_sequences = np.array(pad_sequences(input_sequences, maxlen=tuneTimeSlice, padding='pre'))
  classes = np.amax(input_sequences)+1
  X = []
  X = input_sequences[:,:10]
  y2 = input_sequences[:,-1]
  y = ku.to_categorical(y2,classes)
  return input_sequences,X,y,y2,classes
  
  
#Gives 'length' number of end notes
def composeEndNote(sample,length):
  input = np.array(pad_sequences([[sample]], maxlen=tuneTimeSlice - 1, padding='pre'))[0]
  composition = getEnding(input,length)
  composition = np.insert(composition,0,sample)
  return composition

#Gives prediction  based on end note
def composeTune(sample,length):
  tunesample=[]
  for i in range(0,len(sample)):
     tunesample.append([sample[i]])  
  tunesample = pad_sequences(tunesample, maxlen=tuneTimeSlice-tuneOutputSpan, padding='pre')
  composition = [] 
  for endnote in tunesample:
    composition += [generateResolvent(endnote,length)]
  return composition


#Haritha's stuff
def generateResolvent(Notes,length):
  output=[]
  length = length / tuneOutputSpan
  if tuneReverse == True:
    output = pad_sequences([np.flip(Notes[list(Notes).count(0):])], maxlen=len(Notes), padding='pre')[0]
  for i in range(0,length):
    predArray =  list(modeltune.predict_proba([[list(Notes)]], verbose=0)[0])
    pred = nMostProbablePredictions(variety,predArray)[int(random() * tuneTemperature)]
    pred = intervalToIndex(pred)
    output = np.append(output,pred)
    Notes = np.append(Notes,pred)
    Notes = np.delete(Notes,range(0,tuneOutputSpan))
  if tuneReverse == True:
    output = pad_sequences([np.flip(output[list(output).count(0):])], maxlen=len(output), padding='pre')[0]
  return output




In [0]:
#@title All Motif Functions




def findSimilarityScore(a,b):
  score = 0
  #Remove the empty note in the end
  if abs(sum(a)-sum(b)) >= 2:
    return score
  else:
    aPtr = 0
    aTime = 0
    bPtr = 0
    bTime = 0
    score = 0
    while True:
      if aPtr >= len(a):
        break
      if bPtr >= len(b):
        break
      if a[aPtr] == b[bPtr]:
          score += 1
          aTime = bTime = bTime + b[bPtr]
      else:
        bTime += b[bPtr]
        aTime += a[aPtr]
        if a[aPtr] < b[bPtr]:
          while aTime < bTime and aPtr < len(a) - 1:
            aPtr += 1
            aTime += a[aPtr]
        elif b[bPtr] < a[aPtr]:
          while bTime < aTime and bPtr < len(b) - 1:
            bPtr += 1
            bTime += b[bPtr]
      aPtr += 1
      bPtr += 1
  return float(score)/max(len(a),len(b))


  
  
#Remove duplicate repeating phrases from Phrases
def removeDuplicatePhrases(Phrases):
  encountered = [Phrases[0]]
  for i in range(1,len(Phrases)):
    for j in range(0,len(encountered)):
      deletedFlag = False
      if i >= len(Phrases):
        return Phrases
      score = findSimilarityScore(encountered[j],Phrases[i])
      if score > 0.5:
        del Phrases[i]
        deletedFlag = True
    if deletedFlag != True:
      encountered += [Phrases[i]]
  return Phrases



#@title
#Inputs : input_sequences : List of Notes lengths of varying lenght
#Outputs :  X : Note lengths of length equal to tuneTimeSlice - tuneOutputSpan pre padded with zeros
#           y : Set of output lengths for each X encoded in one hot encoding format
#           intervals : Set of output lengths for each X in readable format
#           tokenizer : word index mapping set of output notes to their uniquely assigned labels
#           classes : Number of possible combination of Output Notes that for y
def generate_padded_sequences_motif(input_sequences):
  temp = []
  for i in input_sequences:
    numberOfZeros = max(0,motifTimeSlice - len(i))
    temp += [list(np.zeros(numberOfZeros,dtype = 'int')) + i]
  input_sequences = temp 
  motifX = []
  intervals = []
  for i in range(0,len(input_sequences)):
    if np.count_nonzero(input_sequences[i][:-motifOutputSpan]) != 0:
      motifX += [list(input_sequences[i][:-motifOutputSpan])]
      interval = str(input_sequences[i][-motifOutputSpan])
      for j in range(-motifOutputSpan + 1 ,0):
          interval += '-' + str(input_sequences[i][j])
      intervals += [interval]
  tokenizer = Tokenizer(filters = '_')
  tokenizer.fit_on_texts(intervals)
  tokenizerInput = str(list(chain.from_iterable(labelledPhrases)))[1:-1].replace(', ','_')
  tokenizer.fit_on_texts(tokenizerInput)
  tokenizer.fit_on_texts(np.array(range(0,stepsPerBar),dtype='string'))
  for i in range(0,len(motifX)):
    for j in range(0,len(motifX[i])):
      if str(motifX[i][j]) != '0':
        motifX[i][j] = int(tokenizer.word_index[str(motifX[i][j])])
  motifX = np.array(motifX,dtype = int)
  y = []
  for i in intervals:
    y = y + [tokenizer.word_index[i]]
  classes = np.max(y)+1
  y = ku.to_categorical(y,classes)
  return motifX,y,intervals,tokenizer,classes




def keyFunction(a):
  return len(a[0])
  
  

#Calls maxSubString for different lengths of motifs
def findMotif(motifStr,maxSizeOfMotif):
    mostfreq = []
    for i in range(1,maxSizeOfMotif + 1):
      d = maxSubString(motifStr,i)     
      temp = sorted(d.items(), key=operator.itemgetter(1), reverse= True)
      for j in temp:
        if j[1] > 0 and j[0].count(',') > 0:
          mostfreq += [j]
    mostfreq = sorted(mostfreq,key = keyFunction)
    i = len(mostfreq) - 1
    while i >= 0:
      j = i - 1
      while j >= 0 and len(mostfreq[i][0]) >= len(mostfreq[j][0]):
        if mostfreq[i][0].find(mostfreq[j][0][1:-1]) >= 0 and mostfreq[i][1] == mostfreq[j][1]:
          del mostfreq[j]
          i -= 1
          j -= 1
        if stepsPerBeat == 3 and len(mostfreq[j][0]) < 7:
          del mostfreq[j]
          i -= 1
          j -= 1
        else:
          j -= 1
      i -= 1
    mostfreq = mostfreq[:26]
    return {item[0] for item in mostfreq}
  
  
#Finds maximum duration subsequence in string representation of Motif
def maxSubString(motifStr,l):
  timeSpan = l * stepsPerBeat
  motifList = []
  for motifArray in motifStr:
    motifArray = motifArray[:-1]
    i = 0
    while i < len(motifArray):
      start = i
      span = motifArray[i]
      if span % timeSpan == 0 :
        i += 1
        break
      elif span > timeSpan :
        span = span % stepsPerBeat
      while span < timeSpan:
        i += 1
        if i >= len(motifArray):
          break
        span += motifArray[i]
      i += 1
      stop = i
      motifList += [motifArray[start:stop]]
  unique, counts = np.unique(motifList, return_counts=True)
  uniqueStrings = [str(x) for x in unique]
  scores = [int((c/2)) - 2 for c in counts]
  return dict(zip(uniqueStrings, scores))

  
  
  
  


def searchMotif(ch):
  if ch in ascii_lowercase:
    return motifDictionary[ch]
  else:
    return ch



def generatenGramSequence(A):
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in A:
      if line[0] != stepsPerBar:
        if motifReverse == True:
          line = np.flip(line)
        for i in range(1, len(line)):
            n_gram_sequence = line[:i+1]
            if len(n_gram_sequence) > motifTimeSlice:
              for i in range(0,len(n_gram_sequence) - motifTimeSlice):
                input_sequences.append(list(n_gram_sequence[i:i+motifTimeSlice]))
            else:
              input_sequences.append(list(n_gram_sequence))
    return input_sequences

  


def encode(Notes,singleElement = False):
  array = []
  for note in Notes:
    if str(note) == '0':
      array += [0]
    else:
      array += [int(motifTokenizer.word_index[str(note)])]
  if singleElement == False:
    numberOfZeros = max(0,motifTimeSlice - len(array))
    array = list(np.zeros(numberOfZeros,dtype = 'int')) + list(array)
  return np.array(array)



  
#@title
def decode(Notes):
  array = []
  for note in Notes:
    if note != '0':
      for word,index in motifTokenizer.word_index.items():
        if str(index) == str(note):
          for i in word.split('-'):
              array += [i]
  return array

      

  
  
def sum2(arr):
  x = []
  for i in arr:
    x += i
  return x




#Finding Pattern among bars by using similarity score
def findPattern(bars):
  i = 0
  pattern = []
  length = len(bars)
  single = double = triplet = quadraple = octet = hexaple = 0
  while i < length - 1:
    single = findSimilarityScore(bars[i],bars[i+1])
    if i < length - 4:
      double = findSimilarityScore(sum2(bars[i:i+2]),sum2(bars[i+2:i+4]))
      if i < length - 6:
        triplet = findSimilarityScore(sum2(bars[i:i+3]),sum2(bars[i+3:i+6]))
        if i < length - 8:
          quadraple = findSimilarityScore(sum2(bars[i:i+4]),sum2(bars[i+4:i+8]))
          if i < length - 16:
            octet = findSimilarityScore(sum2(bars[i:i+8]),sum2(bars[i+8:i+16]))
            if i < length - 32:
              hexaple = findSimilarityScore(sum2(bars[i:i+16]),sum2(bars[i+16:i+32]))
            else :
              hexaple = 0
          else :
            octet = 0
        else :
          quadraple = 0
      else:
        triplet = 0
    else:
      double = 0
    maximum = max(single,double,triplet,quadraple,octet,hexaple,0.7)
    if maximum == 0.7:
      pattern += [bars[i]]
      i += 1
    if maximum == hexaple:
      pattern += [sum2(bars[i:i+16])]
      i += 32
    if maximum == octet:
      pattern += [sum2(bars[i:i+8])]
      i += 16
    if maximum == quadraple:
      pattern += [sum2(bars[i:i+4])]
      i += 8
    elif maximum == triplet:
      pattern += [sum2(bars[i:i+3])]
      i += 6
    elif maximum == double:
      if maximum == findSimilarityScore(sum2(bars[i+2:i+4]),sum2(bars[i+4:i+6])):
        pattern += [sum2(bars[i:i+2])]
        i += 6
      else:
        pattern += [sum2(bars[i:i+2])]
        i += 4
    elif maximum == single and i < length:
      pattern += [bars[i]]
      i += 2
  return pattern

  

def composeGroove(Notes,totalDuration,numberOfNotes = 0):
    if str(Notes) in ascii_lowercase:
      decodedNotes = motifDictionary[Notes].split(', ')
      Notes = encode(decodedNotes,False)[1:]
    else:
      decodedNotes = decode(Notes)  
    totalDuration = totalDuration * stepsPerBeat
    output = decodedNotes
#     soFarDuration = 0
#     for note in decodedNotes:
#       if note[0] == '[':
#         soFarDuration += np.array(note[1:-1].split(','),dtype = int).sum()
#       else:
#         soFarDuration += int(note)    
    if motifReverse == True:
      output = pad_sequences([np.flip(Notes[list(Notes).count(0):])], maxlen=len(Notes), padding='pre')[0]
    attempts = 0
    soFarDuration = sum(output)
#     print(output,sum(output),soFarDuration)
    while soFarDuration < totalDuration:
#       print(output,sum(output),soFarDuration)
      if stepsPerBeat == 4 and soFarDuration % stepsPerBeat == 1 and random() < offByOne:
        pred = motifTokenizer.word_index['3']
      elif stepsPerBeat == 4 and soFarDuration % stepsPerBeat == 3 and random() < offByThree:
        pred = motifTokenizer.word_index['1']
      else:
        predArray =  list(motifModel.predict_proba([[list(Notes)]], verbose=0)[0])
        pred = nMostProbablePredictions(variety,predArray)[int(random() * motifTemperature)]
        
      if len(decode([pred])) != 0:
          note = decode([pred])[0]
          if note.find(',') > -1:
            temp = np.array(note[0].split(', '),dtype = int).sum()
          else:
            temp = int(note)
          if (temp + soFarDuration) <= totalDuration:
            soFarDuration += temp
            output = np.append(output,temp)
            Notes = np.append(Notes,pred)
            Notes = np.delete(Notes,[0])
            
          if attempts > totalDuration:
            pred = motifTokenizer.word_index[str(totalDuration - soFarDuration)]
            output = np.append(output,temp)
            Notes = np.append(Notes,pred)
            Notes = np.delete(Notes,[0])
            break
      attempts += 1 
    if motifReverse == True:
      output = pad_sequences([np.flip(output[list(output).count(0):])], maxlen=len(output), padding='pre')[0]
#     output = np.array(str(decode(output)).replace("'",'').replace('[','').replace(']','').replace(',','').split(),dtype=int)  
    output = np.array(str(output).replace("'",'').replace('[','').replace(']','').replace(',','').split(),dtype=int)  
    output = output[output != 0]
    print(output,soFarDuration)
    if numberOfNotes == 0:
      return output,soFarDuration
    else:
      return output[:numberOfNotes - 1] + [sum(output[numberOfNotes - 1:])],soFarDuration
      
  


In [0]:
#@title Build Motif Model

offByOne = 0
offByThree = 0
for i in offBy:
  if i % 4 == 1 :
    offByOne += 1
  elif i % 4 == 3 :
    offByOne += 3
    
offByOne /= len(offBy)
offByThree /= len(offBy)



Motifs = findMotif(bars,maxSizeOfMotif)


temp = dict()
i = 0
for m in Motifs:
    temp[m[1:-1]] = ascii_lowercase[i]
    i += 1
Motifs = temp

patterns = findPattern(bars)
input_sequences = patterns[:]

averageEndingNoteLength = sum([x[-1] if len(x) > 1 and str(x[-1]) not in ascii_lowercase else 0 for x in patterns])/len(patterns)


labelledPhrases = []
for i in range(0,len(input_sequences)):
  for motif,symbol  in Motifs.items():
    input_sequences[i] = str(input_sequences[i]).replace(motif + ',',symbol + ',')
  labelledPhrases += [input_sequences[i][1:-1].split(', ')]
motifDictionary = dict()
for key,value in Motifs.items():
  motifDictionary[value] = key

startNotes = np.unique([str(x[0]) for x in labelledPhrases])

labelledPhrases = generatenGramSequence(labelledPhrases)


motifX,motif_y,motif_y2,motifTokenizer,classes = generate_padded_sequences_motif(labelledPhrases)

del motifTokenizer.word_index["'"]



#Reverse word index for motifTokenizer
for key,value in motifTokenizer.word_index.items():
  if key.isalpha():
    del motifTokenizer.word_index[key]
    motifTokenizer.word_index[searchMotif(key)] = value 
    
    

input_len = len(motifX[0])
total_words = len(motifX)
motifModel = Sequential()

# Add Input Embedding Layer
motifModel.add(Embedding(total_words, input_len, input_length=input_len))

# Add Hidden Layer 1 - LSTM Layer
motifModel.add(LSTM(25))
motifModel.add(Dropout(0.1))

# Add Output Layer
motifModel.add(Dense(classes, activation='softmax'))

motifModel.compile(loss='categorical_crossentropy', optimizer='adam')
# motifModel.summary()

y_probs = motifModel.fit(motifX, motif_y, epochs=100, verbose=0)


W0711 05:49:27.904669 140460454471552 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0711 05:49:27.920768 140460454471552 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0711 05:49:27.924818 140460454471552 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0711 05:49:28.129476 140460454471552 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0711 05:49:28.140418 

In [0]:
#@title Build Tune model

# maxNote = [max(n) for n in phrasesNotes]
# maxNote = sum(maxNote)/len(maxNote)


# # octaveShift = (int)(maxNote/12) - 2
# octaveShift = (int)(maxNote/12) - 1
# shift = (octaveShift * 12)


# x=[(np.array(x) - shift).tolist() for x in phrasesNotes]

# i = 0
# while i < len(x):
#   if min(x[i])<0:
#     del x[i]
#   i += 1

  
input_sequences = phrasesNotes
input_sequences = generateTuneNGramSequence(input_sequences)

Xtune,ytune,y2tune,tokenizertune,classestune = generate_padded_sequences_notes(input_sequences)



input_len = len(Xtune[0])
total_words = len(Xtune)
modeltune = Sequential()


# Add Input Embedding Layer

modeltune.add(Embedding(total_words, input_len, input_length=input_len))

# Add Hidden Layer 1 - LSTM Layer
modeltune.add(LSTM(100))
modeltune.add(Dropout(0.1))

# Add Output Layer
modeltune.add(Dense(classestune, activation='softmax'))
modeltune.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
y_probs = modeltune.fit(Xtune, ytune, epochs=100,verbose = 0)



In [0]:
#@title Build Endnote model
      
endNotesPhrase = []
for i in range(len(phrasesNotes)):
  endNotesPhrase.append(notes[i][-1])
  
  
# endNotesPhrase = []
# for i in range(len(phrasesNotes)):
#   endNotesPhrase.append(phrasesNotes[i][-1])
  
  
EndNoteInput= [np.array(endNotesPhrase)]    
input_sequences = generateTuneNGramSequence(EndNoteInput)
input_output_sequence,Xend,yend,y2end,classes = generate_padded_sequences_end(input_sequences)

input_len = len(Xend[0])
total_words = len(Xend)
outmodel = Sequential()

# Add Input Embedding Layer
outmodel.add(Embedding(classes, input_len, input_length=input_len))

# Add Hidden Layer 1 - LSTM Layer
outmodel.add(LSTM(100))
outmodel.add(Dropout(0.1))

# Add Output Layer
outmodel.add(Dense(classes, activation='softmax'))
outmodel.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])


outmodel.fit(Xend, yend, epochs=100, verbose=0)

In [0]:
#@title Compose phrase

#Inputs : duration : number of counts of composition
def composePhrase(duration,numberOfNotes = None,groove = None,endnotes = None,terminating = False):
  octave = 4
  if type(groove) == type(None):
    startNote = startNotes[int(random() * len(startNotes))]
    if startNote in ascii_lowercase:
      randomGroove = startNote
    else:
      randomGroove = encode(startNote)[-(motifTimeSlice-1):]
    if numberOfNotes == None:
      groove,duration = composeGroove(randomGroove,duration,0)
    else:
      groove,duration = composeGroove(randomGroove,duration,numberOfNotes + 1)

  if terminating == True:
    groove = list(groove)
    while groove[-1] < averageEndingNoteLength and len(groove) > 2:
        groove[-2:] = [sum(groove[-2:])]
    
  if type(endnotes) == type(None):
    endnotes = composeEndNote(userEndNote,numberOfPhrases - 1)
  tune = composeTune(np.flip(endnotes),len(groove))
#   print(tune)
#   tune = [userPhrase] + tune[1:]
  
  temp = []
  for i in range(0,len(tune)):
    temp += [filter(lambda a: a != 0, tune[i])]
  tune = temp
  
  tune = [[(x + (12 * octave)) for x in tune[0]]]
  
  return numberOfNotes,tune,groove,duration
  
  
  

  
def play(numberOfNotes,tune,groove,duration,tempo):
  timing = (Decimal(60)/Decimal(tempo))*Decimal(0.25)
  MelliHamsa = music_pb2.NoteSequence()
  t = 0
  for j in range(0,len(tune)):
    MelliHamsa.tempos.add(qpm=240)
#     octave = 4
    stepCount = 0
    if numberOfNotes != None:
      tune[j] = tune[j][-numberOfNotes:]
    for i in range(0,min(len(groove),len(tune[j]))):
      notelength = timing * groove[i]
#       note = int(tune[j][i] + 12 * octave)
      note = tune[j][i]
      MelliHamsa.notes.add(pitch = note,start_time = t,end_time = t + notelength,velocity = int(50 + random()*50))
      t += notelength
      stepCount += groove[i]
#     print(duration,stepCount)
    restPeriod = (duration - stepCount) * timing
    MelliHamsa.notes.add(pitch = 0,start_time = t,end_time = t + restPeriod,velocity = int(50 + random()*50))
    t += restPeriod
  MelliHamsa.total_time = t
  mm.plot_sequence(MelliHamsa)
  mm.play_sequence(MelliHamsa,synth=mm.fluidsynth)
  
  
  
  
def play2(tune,groove,tempo):
  timing = (Decimal(60)/Decimal(tempo))*Decimal(0.25)
  MelliHamsa = music_pb2.NoteSequence()
  t = 0
  for j in range(min(len(groove),len(tune))):
    MelliHamsa.tempos.add(qpm=240)
#     octave = 4
    stepCount = 0
    notelength = timing * groove[j]
#     note = int(tune[j] + 12 * octave)
    note = tune[j]
    MelliHamsa.notes.add(pitch = note,start_time = t,end_time = t + notelength,velocity = int(50 + random()*50))
    t += notelength
    stepCount += groove[j]
  MelliHamsa.total_time = t
  mm.plot_sequence(MelliHamsa)
  mm.play_sequence(MelliHamsa,synth=mm.fluidsynth)
  

In [0]:
#@title Pattern Module Functions

def splitBar(bars,duration,fromEnd = False):
  
  Sum = 0
  barLength = 0
  for i in range(len(bars)):
    if str(bars[i]) in ascii_lowercase:
      bars[i] = 0
  if(len(bars) == 0):
    return bars
  print(bars)
  if fromEnd:
      Sum += bars[-(barLength + 1)]
  else:
      Sum += bars[barLength]
  while Sum < duration and barLength < len(bars) - 1:
    barLength += 1
    if fromEnd:
      Sum += bars[-(barLength + 1)]
    else:
      Sum += bars[barLength]
  if fromEnd:
      return bars[-(barLength + 1):]
  else:
      return bars[:barLength + 1:]



def endNoteScore(bars):
  npArray = np.array(bars)
  lengthyNotesPositions = np.where(npArray > 4)[0]
  if lengthyNotesPositions.size == 0:
    return 0,0
  barsLength =  len(bars)
  barLength = len(splitBar(bars,duration = stepsPerBar,fromEnd = True))
  endNote = lengthyNotesPositions[-1]
  endNoteScore = (float(lengthyNotesPositions[-1]) / barLength) * barsLength
  return endNote,endNoteScore


def addPattern(bar,duration):
  for k in patternDict.keys():
    if findSimilarityScore(patternDict[k],bar) > 0.8:
      return k
  patternDict[ascii_lowercase[len(patternDict) % 26]+str(duration) + '.'] = bar
  return ascii_lowercase[len(patternDict) - 1]





In [0]:
#@title Pattern Module


#Assign alphabet for each unique pattern
patternDict = dict()
phrase = []
for bar in bars:
  if(sum(bar) == stepsPerBar):
    phrase += bar
    firstHalf = splitBar(bar,stepsPerBar/2)
    secondHalf = splitBar(bar,stepsPerBar/2,True)  
    if findSimilarityScore(firstHalf,secondHalf) > 0.5:
      alphabet = addPattern(firstHalf,2)
    else:
      alphabet = addPattern(bar,4)
    endNote,score = endNoteScore(phrase)
    if score > 0.5:
      phrase = []


temp = dict()
for key in patternDict.keys():
  temp[key] =  str(patternDict[key])[1:-1]
patternDict = temp





#Replace patterns with alphabets
patterns = findPattern(bars)

tempPattern = []
for pattern in patterns:
  temp = str(pattern)
  for key in patternDict.keys():
      temp = temp.replace(patternDict[key],key)
  tempPattern += [temp]
patterns = tempPattern



#Convert to list of tuples
for i in range(len(patterns)):
  patterns[i] = [(j[0],int(j[1])) for j in patterns[i][1:-1].split(', ')]

In [0]:
#@title Compose Function

# compose(duration = 8,tempo= 100,numberOfNotes=LenOfPhrase)

# play(numberOfNotes,tune,groove,duration,tempo = 80)
def compose(inputGroove = None):
#   pattern = patterns[int(random() * len(patterns))]
  pattern = [('a',4)]
  patternDict = dict()
  completeTune = []
  completeGroove = []
  organisedGroove = []
  terminating = False
  
  i = 0
  if type(inputGroove) == type(None):
    numberOfPhrases = len(pattern)
    endnotes = composeEndNote(userEndNote,numberOfPhrases - 1)
    for alphabet,duration in pattern:
      if i == len(pattern) - 1:
        terminating = True
      if alphabet not in patternDict.keys():
        numberOfNotes,tune,groove,duration = composePhrase(duration,endnotes = [endnotes[i]],terminating = terminating)
        patternDict[alphabet] = groove
      else:
        gr = patternDict[alphabet]
        numberOfNotes,tune,groove,duration = composePhrase(duration,groove = gr,endnotes = [endnotes[i]],terminating = terminating)
    #   completeTune += [tune]
    #   completeGroove += [groove]
      completeTune += tune[0]
      completeGroove += list(groove)
      organisedGroove += [list(groove)]
      i += 1
  else:
    numberOfPhrases = len(inputGroove)
    endnotes = composeEndNote(userEndNote,numberOfPhrases - 1)
    for i in range(len(inputGroove)):
      if i == len(pattern) - 1:
        terminating = True
      duration = sum(inputGroove[i])/4
      numberOfNotes,tune,groove,duration = composePhrase(duration,groove = inputGroove[i],endnotes = [endnotes[i]],terminating = terminating)
    #   completeTune += [tune]
    #   completeGroove += [groove]
      completeTune += tune[0]
      completeGroove += list(groove)
      organisedGroove += [list(groove)]
      i += 1
  return completeTune,completeGroove,organisedGroove

In [0]:
#@title Compose with User Input
numberOfNotes = len(userPhrase)
completeTune1,completeGroove,organisedGroove = compose()
completeGroove = completeGroove[:numberOfNotes - 1] + [sum(completeGroove[numberOfNotes - 1:])]
completeTune1 = list(userPhrase)
octave = 4
completeTune1 = [(x + (12 * octave)) for x in completeTune1]
completeTune2,completeGroove,organisedGroove = compose([completeGroove])
completeTune3,completeGroove,organisedGroove = compose(organisedGroove)
completeTune4,completeGroove,organisedGroove = compose(organisedGroove)
play2(completeTune1 + completeTune2 + completeTune3 + completeTune4,completeGroove * 4,tempo = 100)

(array([2, 2, 2, 4, 7]), 15)


Loading BokehJS ...

In [0]:
#@title Compose without User Input
completeTune1,completeGroove,organisedGroove = compose()
completeTune2,completeGroove,organisedGroove = compose(organisedGroove)
completeTune3,completeGroove,organisedGroove = compose(organisedGroove)
completeTune4,completeGroove,organisedGroove = compose(organisedGroove)
play2(completeTune1 + completeTune2 + completeTune3 + completeTune4,completeGroove * 4,tempo = 80)

(array([2, 5, 5, 5]), 15)
[2 5 5 5]
[2, 5, 5, 5]
[2, 5, 5, 5]
[2, 5, 5, 5]


Loading BokehJS ...

In [0]:
#@title Tune to CSV
def tuneToCsv(inputFileName,outputFileName,tune,groove,clocksPerStep = 24):
  #Retrieving header files from the original
  MIDI_signals = [' Header', ' Time_signature', ' Tempo', ' Title_t',' Start_track\n']
  headerLines = []
  inputFile = open(inputFileName,'r')
  for line in inputFile.readlines():
    columns = line.split(',')
    if len(columns) > 2 and columns[2] in MIDI_signals:
      MIDI_signals.remove(columns[2])
      headerLines += [line]
      if columns[2] == ' Title_t':
        channel = int(columns[0])

  #Get clock values
  clock = 0
  notes = []
  clocks = []
  for i in range(len(groove)):
    notes += [tune[i],tune[i]]
    clocks += [clock,clock + groove[i] * clocksPerStep]
    clock += groove[i] * clocksPerStep

  #Write rows to csv  
  outputFileName = 'MelliCompose.csv'
  outputFile =  open(outputFileName,'w')
  outputFile.writelines(headerLines)
  for i in range(0,len(clocks),2):
    velocity = int(60 + (67 * random()))
    row = [str(channel),str(clocks[i]),'Note_on_c','0',str(notes[i]),str(velocity)]
    outputFile.write(', '.join(row) + '\n')
    row = [str(channel),str(clocks[i + 1]),'Note_off_c','0',str(notes[i + 1]),str(velocity)]
    outputFile.write(', '.join(row) + '\n')

  row = [str(channel),str(clocks[-1]),'End_track']
  outputFile.write(', '.join(row) + '\n')
  outputFile.write('0, 0, End_of_file')
  outputFile.close()


In [0]:
#@title Call Tune to csv
tune = completeTune1 + completeTune2 + completeTune3 + completeTune4
groove = completeGroove * 4
tuneToCsv(inputFileName,outputFileName,tune,groove,clocksPerStep)
files.download(outputFileName)